In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from argparse import Namespace
import matplotlib.pyplot as plt
import av2.rendering.vector as vector_plotting_utils
from shapely.geometry import Polygon

from graph_creator.MapGraph import MapGraph
from graph_creator.ActorGraph import ActorGraph

from av2.map.map_api import ArgoverseStaticMap
from av2.datasets.motion_forecasting import scenario_serialization

from av2.datasets.motion_forecasting.viz.scenario_visualization import (
    visualize_scenario
)
from av2.map.map_api import ArgoverseStaticMap
import os
import pickle
from tqdm import tqdm
import networkx as nx
os.getcwd()


from graph_creator.create_graph import get_scenario_data, plot_scene_at_timestep

In [4]:
from pathlib import Path
# repo_root = Path("/Users/marius/code/graph_coverage")
repo_root = Path.cwd()
while not (repo_root / ".git").exists() and repo_root != repo_root.parent:
    repo_root = repo_root.parent
dataroot = repo_root / "argoverse_data" / "train"
print(repo_root)


/Users/marius/code/graph_coverage


In [5]:
scenario_folders = [f for f in dataroot.iterdir() if f.is_dir()]

# Print number of scenarios found
print(f"Found {len(scenario_folders)} scenarios")

Found 199908 scenarios


In [6]:
# @ Thomas run here for data creation for one scenario:

i = 1001
scenario = scenario_folders[i]
log_id = scenario.name
# check if file exists:
timestamp = 1.0
print(i, log_id)
scenario, map = get_scenario_data(dataroot, log_id)
G_map = MapGraph.create_from_argoverse_map(map)
actor_graph = ActorGraph.from_argoverse_scenario(
                                scenario, 
                                G_map, 
                                delta_timestep_s=1.0,
                                max_distance_lead_veh_m=50,
                                max_distance_opposite_forward_m=100,
                                max_distance_opposite_backward_m=10,
                                max_distance_neighbor_forward_m=50,
                                max_distance_neighbor_backward_m=20,
                                max_node_distance_leading=10,
                                max_node_distance_neighbor=6,
                                max_node_distance_opposite=6,
)

timestamps = list(actor_graph.actor_graphs.keys())
timestamp = timestamps[2]
actor_graph.actor_graphs[timestamp]
# check if graph is connected:
# nx.is_connected(actor_graph.actor_graphs[timestamp]) # not implemented for directed graphs... 
#dir(actor_graph)

actor_graph.actor_components[timestamp]

keys = list(actor_graph.actor_graphs.keys())
for  key in keys:
    actor_graph.actor_components[key] = [actor_graph.actor_components[key][i] for i in range(len(actor_graph.actor_components[key])) if actor_graph.actor_components[key][i].size() > 1]

for timestamp in timestamps:
    for component_idx in range(len(actor_graph.actor_components[timestamp])):
        # save_path = f'/home/tmuehlen/repos/graph_coverage/actor_graphs/argoverse_components/graph_{log_id}_{timestamp}_{component_idx}.pkl'
        save_path = f'/home/tmuehlen/repos/graph_coverage/actor_graphs/argoverse_components_nx/graph_{log_id}_{str(timestamp).replace(".", "_")}_{component_idx}.pkl'
        # with open(save_path, "wb") as file:
        #     pickle.dump(actor_graph.actor_components[timestamp][component_idx], file)

1001 00a26d3e-4d90-4ff9-87b9-d6e5903dc9ab


In [11]:
# @ Thomas run here for data creation
actor_graphs = []

for i, scenario in enumerate(scenario_folders):
    if i >= 1000:
        break
    log_id = scenario.name
    # check if file exists:
    # if (repo_root / "actor_graphs" / "argoverse" / f"{log_id}_map_graph.pkl").exists():
    #     continue
    timestamp = 1.0
    print(i, log_id)
    scenario, map = get_scenario_data(dataroot, log_id)
    G_map = MapGraph.create_from_argoverse_map(map)
    actor_graph = ActorGraph.from_argoverse_scenario(
                                    scenario, 
                                    G_map, 
                                    delta_timestep_s=1.0,
                                    max_distance_lead_veh_m=50,
                                    max_distance_opposite_forward_m=100,
                                    max_distance_opposite_backward_m=10,
                                    max_distance_neighbor_forward_m=50,
                                    max_distance_neighbor_backward_m=20,
                                    max_node_distance_leading=10,
                                    max_node_distance_neighbor=6,
                                    max_node_distance_opposite=6,
    )
    # actor_graphs.append(actor_graph)
    #actor_graph.visualize_actor_graph(t_idx=timestamp, comp_idx=0,  save_path=(repo_root / "plots" / f"{log_id}_actor_graph.png"))
    #visualize_scenario(scenario, map, save_path=(repo_root / f"{log_id}_video_.mp4") )
    #plot_scene_at_timestep(scenario, map, timestamp, save_path=(repo_root / "plots" / f"{log_id}_scene_at_{timestamp}.png"), lane_label=True, actor_graph=actor_graph)
    #G_map.visualize_graph(save_path=(repo_root / "plots" / f"{log_id}_map_graph.png"))
    with open(repo_root / "actor_graphs" / "argoverse" / f"{log_id}_map_graph.pkl", "wb") as f:
        pickle.dump(G_map, f)
    with open(repo_root / "actor_graphs" / "argoverse" / f"{log_id}_actor_graph.pkl", "wb") as f:
        pickle.dump(actor_graph, f)
    
    keys = list(actor_graph.actor_graphs.keys())
    for  key in keys:
        actor_graph.actor_components[key] = [actor_graph.actor_components[key][i] for i in range(len(actor_graph.actor_components[key])) if actor_graph.actor_components[key][i].size() > 1]

    for timestamp in timestamps:
        for component_idx in range(len(actor_graph.actor_components[timestamp])):
            save_path = f'{repo_root}/argoverse_components_nx/graph_{log_id}_{str(timestamp).replace(".", "_")}_{component_idx}.pkl'
            with open(save_path, "wb") as file:
                pickle.dump(actor_graph.actor_components[timestamp][component_idx], file)

   

0 000ace8b-a3d2-4228-bd87-91b66a9c5127
1 d25d1aaa-8bb2-4c6b-98a5-fa5aa2ddd2eb
2 5758074f-be16-49da-8bb5-d43a0d8cd034
3 9688c3e8-525c-4557-a60f-a64ad6f18b55
4 7ad01855-0735-482d-8ee2-9ee913f4db09
5 57f6a0cb-d249-49b7-b78d-48e060c426c9
6 129c86f2-9f75-4230-bd39-5c53d96d3512
7 b70ec102-fe92-4d08-8eb7-820c25847db4
8 9665dcb4-5533-4f91-bb90-e31164f162dd
9 8308baf0-fd9a-4cfe-b3f9-7666b47f7da7
10 e8cb4786-c272-4586-9fb3-5e014361c9de
11 7e756b1c-3d34-4490-bcae-636045b241d5
12 7f14ce88-18e3-4e85-aa6c-36cba06bffeb
13 f4c851e8-6a87-44f3-bd6a-283a502e04f4
14 b0aa022b-1ed7-4db4-93b7-3b9ec09184e4
15 f1532d28-88b2-45c3-a689-1673968cfb18
16 346c5833-8ad4-4fba-97df-08ed7117a8fc
17 950f24bb-056c-4b44-9c94-26a917a68e36
18 0ca1767f-c5e1-45f9-b63d-734e470d9003
19 a3a43e5a-4fbe-4e40-9447-21bc2dc666a7
20 b8faf2b6-f1db-4a74-b409-a464f31e3d39
21 f0b19dce-69df-4b2a-92fa-1b91730d9229
22 d45ce3f8-1dcd-491f-9080-0c21cc9827ab
23 2c202eb5-67c7-4805-b28c-31e57f9dce13
24 479403af-1d01-4c99-84a2-e78fa60e58ac
25 cb55173

In [ ]:
# # and now split the graphs into networkx data:

# argoverse_scns = os.listdir("/home/tmuehlen/repos/graph_coverage/actor_graphs/argoverse")
# argoverse_scns = [scn.split("_")[0] for scn in argoverse_scns if scn.endswith("_actor_graph.pkl")]
# print(argoverse_scns[:3])

# for scn in tqdm(argoverse_scns):
#     with open(f"/home/tmuehlen/repos/graph_coverage/actor_graphs/argoverse/{scn}_actor_graph.pkl", "rb") as f:
#         actor_graph = pickle.load(f)
#     timestamps = list(actor_graph.actor_graphs.keys())
#     for timestamp in timestamps:
#         # actor_graph.visualize_actor_graph(timestamp, 0)
#         save_path = f'/home/tmuehlen/repos/graph_coverage/actor_graphs/argoverse_nx/graph_{scn}_{str(timestamp).replace(".", "_")}.pkl'
#         with open(save_path, "wb") as file:
#             pickle.dump(actor_graph.actor_graphs[timestamp], file)
        


In [ ]:
# actor_graphs = []
# import pickle

# for i, scenario in enumerate(scenario_folders):
#     if i >= 5:
#         break
# #if True: 
#     #log_id = "d25d1aaa-8bb2-4c6b-98a5-fa5aa2ddd2eb"
#     log_id = scenario.name
#     print(log_id)
#     timestamp = 1.0
#     scenario, map = get_scenario_data(dataroot, log_id)
#     G_map = MapGraph.create_from_argoverse_map(map)
#     actor_graph = ActorGraph.from_argoverse_scenario(
#                                     scenario, 
#                                     G_map, 
#                                     delta_timestep_s=1.0,
#                                     max_distance_lead_veh_m=50,
#                                     max_distance_opposite_forward_m=100,
#                                     max_distance_opposite_backward_m=10,
#                                     max_distance_neighbor_forward_m=50,
#                                     max_distance_neighbor_backward_m=20,
#                                     max_node_distance_leading=10,
#                                     max_node_distance_neighbor=6,
#                                     max_node_distance_opposite=6,
#     )
#     actor_graphs.append(actor_graph)
#     actor_graph.visualize_actor_graph(t_idx=timestamp, comp_idx=0,  save_path=(repo_root / "plots" / f"{log_id}_actor_graph.png"))
#     #visualize_scenario(scenario, map, save_path=(repo_root / f"{log_id}_video_.mp4") )
#     plot_scene_at_timestep(scenario, map, timestamp, save_path=(repo_root / "plots" / f"{log_id}_scene_at_{timestamp}.png"), lane_label=True, actor_graph=actor_graph)
#     G_map.visualize_graph(save_path=(repo_root / "plots" / f"{log_id}_map_graph.png"))
#     #with open(repo_root / "data" / f"{log_id}_map_graph.pkl", "wb") as f:
#     #    pickle.dump(G_map, f)
#     #with open(repo_root / "data" / f"{log_id}_actor_graph.pkl", "wb") as f:
#     #    pickle.dump(actor_graph, f)


In [ ]:
import pickle
# Save actor graphs and map data to pickle files
output_dir = repo_root / "data"
output_dir.mkdir(exist_ok=True)

# Save actor graphs
actor_graphs_file = output_dir / "actor_graph_argoverse.pkl"
with open(actor_graphs_file, "wb") as f:
    pickle.dump(actor_graphs, f)

# Save map data for each scenario
map_file = output_dir / f"map_argoverse.pkl"
with open(map_file, "wb") as f:
    pickle.dump(G_map, f)



In [ ]:
from subgraphs.SubgraphExtractor import SubgraphExtractor
import os

# Example usage of the SubgraphExtractor
def analyze_actor_graphs(actor_graphs, strategy='frequency', output_dir=None):
    """
    Analyze actor graphs to find common subgraphs using the specified strategy.
    
    Args:
        actor_graphs: List of ActorGraph objects
        strategy: Strategy for selecting subgraphs
        output_dir: Directory to save visualizations
    
    Returns:
        SubgraphExtractor instance
    """
    
    # Initialize extractor with specified strategy
    extractor = SubgraphExtractor(
        min_subgraph_size=3,
        max_subgraph_size=5, 
        # Custom matchers can be defined here if needed
    )
    
    # Extract subgraphs
    subgraph_library = extractor.extract_subgraphs(actor_graphs[:10])
    
    # Print properties
    extractor.print_subgraph_properties()
    
    # Visualize subgraphs
    if output_dir is not None:
        strategy_dir = os.path.join(output_dir, f"strategy_{strategy}")
        extractor.visualize_subgraphs(output_dir=strategy_dir)
    else:
        extractor.visualize_subgraphs()
    
    return extractor

# Create visualization output directory
output_dir = repo_root /  "plots" / "subgraph_analysis"
os.makedirs(output_dir, exist_ok=True)

# Analyze with different strategies
strategies = ['frequency', 'density', 'coverage', 'complexity', 'size']
extractors = {}

for strategy in strategies:
    extractors[strategy] = analyze_actor_graphs(actor_graphs, strategy, output_dir)

# Example of how to access the subgraphs for a specific actor graph and timestamp
print("\nExample of subgraph decomposition:")
print("=================================")
example_graph_idx = 0  # First graph in the list
example_timestamp = list(actor_graphs[example_graph_idx].actor_graphs.keys())[0]
print(f"Graph {example_graph_idx}, Timestamp {example_timestamp}:")

for sg_info in actor_graphs[example_graph_idx].actor_subgraphs[example_timestamp]:
    sg_id = sg_info['subgraph_id']
    nodes = sg_info['nodes']
    print(f"  Subgraph ID: {sg_id}, Nodes: {nodes}")
    # Additional information can be accessed from sg_info['node_mapping']